### - 데이터 수집

#### # 최근 6개월 동안의 연수 후기(제목, 연수 분야, 별점, 후기, 날짜, 시간)

In [62]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.common.exceptions import TimeoutException, NoSuchElementException, NoSuchWindowException, WebDriverException
import datetime

pd.set_option('display.max_rows', None)

driver = webdriver.Chrome()
chain = ActionChains(driver)

page_num = 1

title = []
field = []
time_list = []
star = []
review = []
day = []

result = pd.DataFrame()

# 현재 날짜와 6개월 전 날짜 계산
current_date = datetime.datetime.now()
six_months_ago = current_date - datetime.timedelta(days=180)

while True:
    url = f'https://teacher.i-scream.co.kr/help/afterword/list.do?searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&searchPrgrTimeArr=&searchCrsLCodeList=&pageIndex={page_num}&sso=ok'
    driver.get(url)
    
    page_has_data = False
    
    # 페이지 내 모든 리뷰 수집
    for num in range(1, 11):
        try:
            # 날짜 추출 및 형식 변환
            d = driver.find_element(By.XPATH, f'//*[@id="content_list"]/div[3]/ul/li[{num}]/div[1]/div[2]/span[2]').text
            review_date = datetime.datetime.strptime(d, '%Y-%m-%d')
            
            # 6개월 이내의 리뷰만 추출
            if review_date >= six_months_ago:
                page_has_data = True
                day.append(d)
                
                # 후기
                r = driver.find_element(By.XPATH, f'//*[@id="content_list"]/div[3]/ul/li[{num}]/div[2]/div[2]/p').text
                review.append(r)
                
                # 분야
                f = driver.find_element(By.XPATH, f'//*[@id="content_list"]/div[3]/ul/li[{num}]/div[1]/div[1]').text.split('/')[1].split(']')[0]
                field.append(f)
                
                # 차시
                ti = driver.find_element(By.XPATH, f'//*[@id="content_list"]/div[3]/ul/li[{num}]/div[1]/div[1]').text.replace('[', '').split('차시/')[0]
                time_list.append(ti)
    
                # 제목
                t = driver.find_element(By.XPATH, f'//*[@id="content_list"]/div[3]/ul/li[{num}]/div[1]/div[1]').text
                t = t[t.find(']') + 2:]
                title.append(t)
                
                # 별점
                stars = driver.find_elements(By.XPATH, f'//*[@id="content_list"]/div[3]/ul/li[{num}]/div[1]/div[2]/div')
                for rr in stars:
                    stars = rr.find_elements(By.CLASS_NAME, 'star.checked')
                    star.append(len(stars))
        
        except NoSuchElementException:
            continue
    
    # 만약 페이지에 데이터가 없으면 반복 종료
    if not page_has_data:
        break
    
    page_num += 1
    
res = pd.DataFrame([title, field, time_list, review, star, day], index=['제목', '연수 분야', '시간', '후기', '별점', '날짜'])
result = pd.concat([result, res], axis=1, ignore_index=True)

result_f = result.T

In [63]:
result_f

,제목,연수 분야,시간,후기,별점,날짜
0,그림으로 배우는 한국사 이미지 노트 2,교과지도,30,역사를 재미있게 자세히 복습할 수 있었습니다.,5,2024-06-14
1,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,쏭쌤과 이종대왕 샘! 너무 대단하시고 감사합니다. 평소에 영상으로 접하다가 알짜배기...,5,2024-06-13
2,다양한 재료로 쉽게 따라 하는 캘리그라피,자기계발,15,말 그대로 글씨 연습이네요. 많이 연습해야 하는 연수입니다. 글씨를 쓰는 동안 마음...,5,2024-06-13
3,"기초학력교육, 관계형성이 먼저다.",학습지도,15,"기초학력 교육이라 그래서, 이미 알고 있는 내용을 가르칠 것이라 생각해서 별 기대없...",5,2024-06-13
4,시크릿 플레이리스트 슈퍼콘서트,자기계발,30,요즘 교육적 이슈가 되고 문해력에 대해서 막연히 알고 있던 것을 넘어서 바르게 이해...,5,2024-06-13
5,일본 현지 성우와 함께 배우는 마법의 360문장,어학,30,다양한 문장을 통해 어휘와 문법을 자세히 배울 수 있어 유익하고 보람있는 일본어 연...,5,2024-06-13
6,이야기가 있는 유쾌한 클래식,자기계발,30,강의 형식이 아니라 편안한 이야기를 듣는 느낌이라 좋았습니다.,5,2024-06-13
7,고전읽기로 1년 학급운영 쉽게 끝내기,학급경영,30,고전읽기 연수를 통해 새롭게 고전을 되돌아 보게 된 기회를 가졌으며 나름대로 다시 ...,5,2024-06-13
8,"어서 와! 이탈리아는 처음이지, 본격 이탈리아 도시 여행",자기계발,15,편안하게 들을 수 있는 연수입니다.,5,2024-06-13
9,"미래를 여는 교육, 2022 개정 교육과정(초등편)",학습지도,15,마음먹고 연수를 듣지 않는 한 바뀐 교과서를 토대로 대략적인 교육과정만 알게 되는데...,5,2024-06-13


#### # 직무연수 과정 목록(제목, 연수 분야, 가격)

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import time
# 웹 드라이버 실행 (Chrome)
driver = webdriver.Chrome()
driver.get("https://teacher.i-scream.co.kr/course/crs/creditList.do?searchOrdinalTyCode=TY01&searchOrderField=NEW&sso=ok")
# 데이터를 담을 리스트 초기화
data = []
try:
    num = 1  # 첫 번째 요소부터 시작
    while True:
        # 현재 페이지의 HTML 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # 각 항목의 링크 가져오기
        xpath = f'//*[@id="content"]/ul/li[{num}]/a'
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, xpath))
        )
        link = element.get_attribute('href')
        # 새 탭에서 링크 열기
        driver.execute_script(f"window.open('{link}', '_blank');")
        driver.switch_to.window(driver.window_handles[-1])
        try:
            # 정보 추출
            field = driver.find_element(By.CSS_SELECTOR, 'div.prs_info span:nth-child(2)').text
            title = driver.find_element(By.CSS_SELECTOR, 'span.tit').text
            price = driver.find_element(By.CSS_SELECTOR, 'span.org.bold.mR0.ft16').text
            # 데이터 추가
            data.append({
                '연수 분야': field,
                '제목': title,
                '가격': price
            })
            print(f"연수 분야: {field}")
            print(f"제목: {title}")
            print(f"가격: {price}")
            print("=" * 30)
        except Exception as e:
            print(f"에러 발생: {str(e)}")
        # 현재 탭 닫기
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        num += 1  # 다음 요소로 넘어감
        # 30개의 항목마다 더보기 버튼 클릭
        if num % 30 == 0:
            try:
                more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, 'divMore'))
                )
                more_button.click()
                time.sleep(2)  # 목록이 로드될 때까지 충분한 시간을 줍니다. 필요에 따라 조정 가능
            except Exception as e:
                print(f"더보기 버튼을 클릭할 수 없습니다. 에러 발생: {str(e)}")
                break  # 더 이상 더보기 버튼이 없으면 종료
except Exception as e:
    print(f"에러 발생: {str(e)}")
finally:
    # 웹 드라이버 종료
    driver.quit()
# 데이터프레임 생성
df = pd.DataFrame(data)
print(df)

연수 분야: 자기계발
제목: 대한민국 핫플! 다시 발견하다. 국내 여행 TOP 120
가격: 85,000원
연수 분야: 어학
제목: 야나두 스르르 학습지 - 기초 영어 편
가격: 85,000원
연수 분야: 자기계발
제목: 맘껏 먹고 가볍게 살기 with 채소과일식
가격: 50,000원
연수 분야: 자기계발
제목: 랜선 음악 여행, 이탈리아에 숨겨진 클래식
가격: 50,000원
연수 분야: ICT
제목: 교사를 위한 구글 활용법(실전편)
가격: 50,000원
연수 분야: 자기계발
제목: 선생님을 따라 해봐요! 교실에서 함께 접는 참 쉬운 종이접기
가격: 50,000원
연수 분야: 자기계발
제목: 어른의 일기
가격: 50,000원
연수 분야: 생활지도
제목: 정신과 전문의 김학철의 교사 마음 치유
가격: 50,000원
연수 분야: 학습지도
제목: 미래교육의 또 다른 이름 다문화 교육
가격: 85,000원
연수 분야: 학습지도
제목: 교실 속 생태환경 수업
가격: 50,000원
연수 분야: 생활지도
제목: 학교에서 꼭 필요한 진로교육
가격: 85,000원
연수 분야: 학습지도
제목: 미래를 여는 교육, 2022 개정 교육과정(초등편)
가격: 50,000원
연수 분야: 학습지도
제목: 수업에 바로 써먹는 문해력 도구
가격: 50,000원
연수 분야: 학습지도
제목: 모두를 위한 교육, 2022 개정 교육과정(중등편)
가격: 50,000원
연수 분야: ICT
제목: OA의 신! 학교에서 쓰는 쉬운 엑셀 + 쉬운 PPT
가격: 140,000원
연수 분야: 자기계발
제목: 한석준의 스피치
가격: 50,000원
연수 분야: 생활지도
제목: 나를 지키는 최소한의 언어습관 선 넘는 당신에게 "할 말은 합니다!"
가격: 50,000원
연수 분야: 학습지도
제목: 융합적 사고 UP! 문해력과 기초학력을 높이는 독서 토론 논술
가격: 85,000원
연수 분야: 생활지도
제목: 수진샘의 따뜻한 관계 맺기
가격: 85,000원
연수 분야: ICT
제목: SW&AI 코딩수업의 모든 것 -

연수 분야: 어학
제목: [700+목표] 한 권으로 끝내는 해커스 토익 700+LC ②
가격: 85,000원
연수 분야: 학습지도
제목: 미래 인재를 위한 디지털 리터러시
가격: 50,000원
연수 분야: 자기계발
제목: 민화, 복을 담아 그리는 우리 그림
가격: 50,000원
연수 분야: 자기계발
제목: 뭉크씨, 도파민 과잉입니다
가격: 50,000원
연수 분야: 자기계발
제목: 교사의 디지털 리터러시를 키우는 IT 상식
가격: 50,000원
연수 분야: 생활지도
제목: 7대 안전 + 감염병! 한 번에 듣는 학교 안전교육
가격: 85,000원
연수 분야: 학습지도
제목: 챗GPT 융합교육 가이드, 인공지능(AI) 챗봇 수업
가격: 50,000원
연수 분야: 어학
제목: [캠브리지] English Grammar in Use
가격: 140,000원
연수 분야: 자기계발
제목: 자연에서 하루 쉼표, 캠핑 & 백패킹
가격: 50,000원
연수 분야: 교과지도
제목: 소설 파친코와 썬킴의 한국근현대사
가격: 50,000원
연수 분야: 자기계발
제목: 어서 와! 이탈리아는 처음이지, 본격 이탈리아 도시 여행
가격: 50,000원
연수 분야: 어학
제목: 일본 현지 성우와 함께 배우는 마법의 360문장
가격: 85,000원
연수 분야: 자기계발
제목: 핸드메이드 감성 소품, 리본공예
가격: 50,000원
연수 분야: 자기계발
제목: 송길영의 데이터로 세상 읽기
가격: 50,000원
연수 분야: 자기계발
제목: 지나영의 본질육아
가격: 85,000원
연수 분야: ICT
제목: 챗GPT와 함께하는 HTHT 미래 교육, 디지털리터러시를 높이는 인공지능(AI) 챗봇 제작 수업
가격: 50,000원
연수 분야: 자기계발
제목: 커피로 성공하는 창업 공식(Feat.미래 창업가를 위한 진로 교육)
가격: 50,000원
연수 분야: 자기계발
제목: 어디까지 가봤니? 아는 만큼 보이는 두근두근 세상 모든 여행!
가격: 85,000원
연수 분야: 어학
제목: 친절한 차이티의 중국어 비밀과

연수 분야: 학습지도
제목: 설민석의 한국사능력검정시험[심화]-하
가격: 75,000원
연수 분야: 자기계발
제목: 슬기로운 집사생활을 위한 고양이 상담소
가격: 40,000원
연수 분야: 학습지도
제목: 즐거운 대화, 하브루타로 수업하기
가격: 75,000원
연수 분야: 어학
제목: 김영철, 타일러의 진짜 미국식 영어2
가격: 75,000원
연수 분야: 자기계발
제목: 마음을 이어주는 마법, 타로 이야기
가격: 75,000원
연수 분야: ICT
제목: 엑셀의 신, 학교에서 쓰는 쉬운 엑셀
가격: 75,000원
연수 분야: 학급경영
제목: 비접촉 놀이로 행복 가득 기분 좋은 교실
가격: 40,000원
연수 분야: 학습지도
제목: 설민석의 한국사능력검정시험[심화]
가격: 130,000원
연수 분야: 자기계발
제목: 알아두면 쓸모있는 교직실무
가격: 130,000원
연수 분야: 자기계발
제목: 학생을 살리는 학교공간혁신
가격: 75,000원
연수 분야: 생활지도
제목: 우리 아이 첫 사회성 수업
가격: 75,000원
연수 분야: 어학
제목: 속 시원하게 말하는 리얼 원어민 영어
가격: 75,000원
연수 분야: 어학
제목: 시원스쿨 생기초 말하기 2
가격: 75,000원
연수 분야: 생활지도
제목: 유네스코 아태교육원과 함께하는 세계시민교육
가격: 75,000원
연수 분야: 학습지도
제목: 미술 속 진짜 아이심리 사로잡기
가격: 75,000원
연수 분야: 어학
제목: Essential Grammar in Use
가격: 130,000원
연수 분야: 자기계발
제목: 한국어교원양성과정 Ⅰ
가격: 75,000원
연수 분야: 자기계발
제목: 한국어교원양성과정 Ⅱ
가격: 75,000원
연수 분야: 자기계발
제목: 한국어교원양성과정 Ⅲ
가격: 75,000원
연수 분야: 교과지도
제목: 기초학력의 첫걸음, 수학교육
가격: 40,000원
연수 분야: 자기계발
제목: 단박에 따라하는 사진 영상 레시피
가격: 75,000원
연수 분야: 어학
제목: 웹툰 '마음의소리'로 배우는 영어,

연수 분야: 생활지도
제목: 신영일과 함께하는 안전교육 365
가격: 40,000원
연수 분야: 자기계발
제목: 미래교육 명견만리
가격: 40,000원
연수 분야: ICT
제목: 알고 싶고 파악하고 싶은 고수의 수업 - 보드편
가격: 40,000원
연수 분야: 학습지도
제목: 기초학력의 첫걸음, 한글교육
가격: 40,000원
연수 분야: 교과지도
제목: 실내에서 끝장내는 학생열광 체육놀이
가격: 75,000원
연수 분야: 어학
제목: 일본어 초급 탈출! 마법의 150회화
가격: 75,000원
연수 분야: 자기계발
제목: 토크콘서트 화통 - 화끈한 멘토의 이야기에서 꿈을 디자인하다
가격: 75,000원
연수 분야: 자기계발
제목: 편지 속 이야기로 알아보는 인문학 여행
가격: 75,000원
연수 분야: 학급경영
제목: 허쌤&나쌤의 승승장구 학급경영
가격: 75,000원
연수 분야: ICT
제목: 알고 싶고 파악하고 싶은 고수의 수업-로봇편
가격: 40,000원
연수 분야: 어학
제목: 15초 따라 하기 왕초보 영어
가격: 75,000원
연수 분야: 어학
제목: 오석태의 영어노하우 훔치기
가격: 65,000원
연수 분야: 학습지도
제목: 참쌤스쿨 그림교실, 교사가 최고의 콘텐츠다
가격: 35,000원
연수 분야: 학습지도
제목: 참쌤의 비주얼씽킹 끝판왕
가격: 65,000원
연수 분야: 학습지도
제목: BK선생님의 쉬운 수업레시피
가격: 65,000원
연수 분야: 학습지도
제목: 학습부진, 기초튼튼! 실력쑥쑥!
가격: 35,000원
연수 분야: 학습지도
제목: 학습부진, 천천히 함께 걷기
가격: 35,000원
연수 분야: 자기계발
제목: 교사, 세상에 질문을 던지다.
가격: 35,000원
연수 분야: 자기계발
제목: 교사 공감 다큐, 나는 꿈꾸는 교사입니다
가격: 35,000원
연수 분야: 학습지도
제목: 자유학기제를 넘어 자유학년제의 수업을 보다
가격: 65,000원
연수 분야: 학습지도
제목: Talk Talk(똑똑)한 금융교육
가격: 35,000원
연수 분야

In [64]:
df

,연수 분야,제목,가격
0,자기계발,대한민국 핫플! 다시 발견하다. 국내 여행 TOP 120,"85,000원"
1,어학,야나두 스르르 학습지 - 기초 영어 편,"85,000원"
2,자기계발,맘껏 먹고 가볍게 살기 with 채소과일식,"50,000원"
3,자기계발,"랜선 음악 여행, 이탈리아에 숨겨진 클래식","50,000원"
4,ICT,교사를 위한 구글 활용법(실전편),"50,000원"
5,자기계발,선생님을 따라 해봐요! 교실에서 함께 접는 참 쉬운 종이접기,"50,000원"
6,자기계발,어른의 일기,"50,000원"
7,생활지도,정신과 전문의 김학철의 교사 마음 치유,"50,000원"
8,학습지도,미래교육의 또 다른 이름 다문화 교육,"85,000원"
9,학습지도,교실 속 생태환경 수업,"50,000원"


#### # 최근 6개월동안의 후기 최종 데이터(제목, 연수 분야, 시간, 후기, 별점, 날짜, 가격)

In [65]:
fin_res = pd.merge(result_f,df)
fin_res

,제목,연수 분야,시간,후기,별점,날짜,가격
0,그림으로 배우는 한국사 이미지 노트 2,교과지도,30,역사를 재미있게 자세히 복습할 수 있었습니다.,5,2024-06-14,"65,000원"
1,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,쏭쌤과 이종대왕 샘! 너무 대단하시고 감사합니다. 평소에 영상으로 접하다가 알짜배기...,5,2024-06-13,"85,000원"
2,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,유익한 내용이 많았습니다. 고맙습니다.,5,2024-05-23,"85,000원"
3,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,저경력교사에게 학급운영을 어떻게 해나갈지 큰 그림을 그려볼 수 있는 연수였습니다.,5,2024-05-04,"85,000원"
4,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,유익한 연수였고 교재로도 유튜브로도 많은 도움을 받았습니다.,5,2024-04-18,"85,000원"
5,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,알차고 유익한 내용 감사드립니다!,5,2024-04-02,"85,000원"
6,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,흔들리지 않는 교실을 위한 단호한 학급 운영 새학기 아이들 학급 운영에 많은 도움이...,5,2024-03-11,"85,000원"
7,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,새학기에 불안한 마음이 많이 드는 신규 교사에게 큰 도움이 되는 연수였습니다!! 특...,5,2024-02-28,"85,000원"
8,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,오랜만에 복직해서 힘든 아이들을 만나게 되어 교사로서 너무 힘든 한해를 보냈었는데 ...,5,2024-02-20,"85,000원"
9,흔들리지 않는 교실을 위한 단호한 학급운영,학급경영,30,교사들이 원리를 모르는 게 아닙니다. 그 원리를 어떻게 실제 교실에서 구현하는 지가...,5,2024-02-13,"85,000원"


In [66]:
# 최근 6개월 후기 csv 파일로 저장
fin_res.to_csv('./ice_review.csv', index=False, encoding='utf-8-sig')

---

### - 기초 분석

#### # 연수 과정별 수강 후기 개수

In [81]:
title_counts = fin_res['제목'].value_counts().reset_index()
title_counts.columns = ['제목', '후기 개수']

title_counts

,제목,후기 개수
0,교사를 위한 구글 활용법(입문자편),73
1,"생각을 꺼내는 교육, 이제는 IB 시대",71
2,"가장 쉬운 수업도구, 패들렛과 띵커벨",52
3,"공간의 재발견, 정리의 힘",48
4,큰별쌤 최태성의 별★별한국사,44
5,꼬리에 꼬리를 무는 교권이야기,35
6,신영일과 함께하는 안전교육 365,32
7,한국어교원양성과정 Ⅰ,30
8,"엑셀의 신, 학교에서 쓰는 쉬운 엑셀",30
9,부자 교사의 재테크,29


#### # 연수 분야별 수강 후기 개수

In [83]:
field_counts = fin_res['연수 분야'].value_counts().reset_index()
field_counts.columns = ['연수 분야', '후기 개수']

field_counts

,연수 분야,후기 개수
0,자기계발,715
1,학습지도,519
2,생활지도,396
3,ICT,275
4,어학,222
5,교과지도,196
6,학급경영,111
